<a href="https://colab.research.google.com/github/yogijonasson/Music-to-Visual-Cues-Exploring-How-AI-Can-Communicate-A-Song-Visually/blob/main/Music2Visual_Investigation_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Music2Visuals**
* First, go to Execution Environment and change the execution environment to GPU.

* Connect to the environment.

* Then, run the following pieces of code to install the dependencies.

* Remember to use **your own Google API**.

In [ ]:
!pip uninstall -y torch torchvision torchaudio

!pip install librosa google-generativeai numpy matplotlib diffusers

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.5.1+cu118
Uninstalling torch-2.5.1+cu118:
  Successfully uninstalled torch-2.5.1+cu118
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
ERROR: pip's dependency resolver does not currently take into accou

In [ ]:
import librosa
import librosa.display
import numpy as np
import google.generativeai as genai
from google.colab import files

# For stable difussion
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt

# --- Gemini Setup -------------------------------------------------------------
GOOGLE_API_KEY = # Put your API here
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# --- File upload --------------------------------------------------------------
# Upload your MP3 file. It should be around 15s. A button will appear during
# the execution to upload your file :)
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# --- Audio features -----------------------------------------------------------
y, sr = librosa.load(filename, sr=22050)
tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

# Mel-Frequency Cepstral Coefficients
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
mfcc_mean = np.mean(mfcc, axis=1)  # Take the mean across time


# Format MFCC values correctly
tempo = float(tempo)
mfcc_values = ", ".join([f"{val:.2f}" for val in mfcc_mean])

# --- Prompt creation ----------------------------------------------------------

prompt = f"What emotion can you infer from this musical piece whose values were obtained with Librosa? " \
         f"Tempo: {tempo:.2f} BPM. MFCC values: {mfcc_values}. Provide the answer in 3 nouns or adjectives."

print()
print("-- The prompt is:")
print(prompt)

# --- Gemini response ----------------------------------------------------------

response_gemini = model.generate_content([
    {"role": "user", "parts": [{"text": "You are an expert in music analysis."}]}, # System message as user message part
    {"role": "user", "parts": [{"text": prompt}]} # Your actual user prompt
])

print()
print("-- The response is:")
print(response_gemini.text)

# --- Stable Diffusion ---------------------------------------------------------

print()
print("-- Generating image...")

# Load the Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe.to("cuda")

# Convert the audio features into a textual prompt (or adjust accordingly)
audio_features_description = "abstract art representing a song with the mood " + response_gemini.text

# Generate the image from the audio feature description
generated_image = pipe(audio_features_description).images[0]

# Display the generated image
plt.imshow(generated_image)
plt.axis('off')
plt.show()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

KeyboardInterrupt: 